# Réunir des jeux de données

L’OFSP actualise quotidiennement plusieurs tableaux de données sur le covid, listés [dans cette page](https://www.bag.admin.ch/bag/fr/home/krankheiten/ausbrueche-epidemien-pandemien/aktuelle-ausbrueche-epidemien/novel-cov/situation-schweiz-und-international.html).

Dans ce notebook, on va réunir deux de ces jeux de données.

In [7]:
import pandas as pd

In [8]:
# On charge un fichier détaillé de l’OFSP dans sa version du 3 septembre 2020

df = pd.read_csv('data/Dashboards_1&2_COVID19_swiss_data_pv-2.csv')

# En principe, on pouvait aussi charger la dernière version directement sur le site de l’OFSP.
# Ça ne marche plus (mais vous pouvez essayer, on sait jamais).
# df = pd.read_excel('https://www.bag.admin.ch/dam/bag/fr/dokumente/mt/k-und-i/aktuelle-ausbrueche-pandemien/2019-nCoV/covid-19-basisdaten-fallzahlen.xlsx.download.xlsx/Dashboards_1&2_COVID19_swiss_data_pv.xlsx')

C:\Users\jf\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (1,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Pandas a dû vous donner une erreur selon laquelle «des colonnes ont des types mixtes». Ça veut dire qu’il n’a pas pu identifier clairement quelle colonne contient une date, un entier, des float, etc. On va examiner ça.

In [9]:
# Combien de lignes en tout?
len(df)

304255

In [10]:
df.head()

,replikation_dt,fall_dt,ktn,akl,sex,Geschlecht,Sexe,fallklasse_3,pttoddat,pttod_1
0,2020-09-03 07:29:56,2020-02-24,TI,70 - 79,1,Männlich,homme,1,NaN,0
1,2020-09-03 07:29:56,2020-02-25,AG,20 - 29,1,Männlich,homme,1,NaN,0
2,2020-09-03 07:29:56,2020-02-26,BL,20 - 29,1,Männlich,homme,1,NaN,0
3,2020-09-03 07:29:56,2020-02-26,BS,20 - 29,2,Weiblich,femme,1,NaN,0
4,2020-09-03 07:29:56,2020-02-26,GE,20 - 29,1,Männlich,homme,1,NaN,0


La colonne **replikation_dt** contient la date de mise à jour du fichier. On n’en a pas besoin. Même chose pour la colonne **Geschlecht**, qui est à double.

In [11]:
del df['replikation_dt']
del df['Geschlecht']

Les autres colonnes sont expliquées dans un fichier de l’OFSP, on va leur donner un nom compréhensible…

In [12]:
df.columns = ['date cas', 'canton', 'age', 'genre_num', 'genre', 'nouveaux cas',
       'date décès', 'nouveaux décès']

C’est mieux!

Si on veut être sûr que les colonnes correspondent – par exemple pour un programme entièrement automatisé – on peut aussi utiliser la méthode suivante:

In [13]:
df.rename(columns={
    'fall_dt': 'date cas',
    'ktn': 'canton'}, inplace=True)

In [14]:
df.head()

,date cas,canton,age,genre_num,genre,nouveaux cas,date décès,nouveaux décès
0,2020-02-24,TI,70 - 79,1,homme,1,NaN,0
1,2020-02-25,AG,20 - 29,1,homme,1,NaN,0
2,2020-02-26,BL,20 - 29,1,homme,1,NaN,0
3,2020-02-26,BS,20 - 29,2,femme,1,NaN,0
4,2020-02-26,GE,20 - 29,1,homme,1,NaN,0


In [15]:
df.tail()

,date cas,canton,age,genre_num,genre,nouveaux cas,date décès,nouveaux décès
304250,NaN,ZH,80+,2,femme,0,2020-09-03,0
304251,NaN,ZH,80+,9,manquant,0,2020-09-03,0
304252,NaN,ZH,Unbekannt,1,homme,0,2020-09-03,0
304253,NaN,ZH,Unbekannt,2,femme,0,2020-09-03,0
304254,NaN,ZH,Unbekannt,9,manquant,0,2020-09-03,0


Combien de lignes avec 0 nouveau cas? Et avec 0 nouveau décès?

In [16]:
len(df[ df['nouveaux cas'] == 0 ])

287564

In [17]:
len(df[ df['nouveaux décès'] == 0 ])

303082

On comprend que notre fichier contient en fait deux tableaux, mis à la suite:
* cas confirmés par classe d’âge, genre et canton
* décès par classe d’âge, genre et canton

Dans cet exercice, on s’intéressera uniquement aux cas. L’instruction `.copy()` fait comprendre à Pandas que je ne veux pas enregistrer un sous-ensemble (une «slice») d’un tableau, mais créer un nouveau tableau.

In [18]:
df = df[ df['nouveaux cas'] > 0 ].copy()

On n’a plus besoin des colonnes relatives aux décès.

In [19]:
del df['date décès']
del df['nouveaux décès']

## Filtrer les données

Pour commencer, on va s’intéresser aux cantons romands en créant un sous-ensemble avec la méthode `isin()`. Elle est très pratique pour proposer une liste de valeurs à vérifier.

On travaillera dans la suite du notebook avec la variable **dfr** (df des cantons romands).

In [20]:
cantons_romands = ['VD', 'GE', 'JU', 'VS', 'NE', 'FR']
dfr = df[ df['canton'].isin( cantons_romands ) ]

In [30]:
dfr.head()

,date cas,canton,age,genre_num,genre,nouveaux cas
4,2020-02-26,GE,20 - 29,1,homme,1
8,2020-02-26,JU,50 - 59,1,homme,1
9,2020-02-26,VD,40 - 49,1,homme,1
12,2020-02-27,GE,20 - 29,2,femme,1
13,2020-02-27,GE,30 - 39,1,homme,1


## Regrouper les données
Et si on s’intéressait uniquement aux cantons, et pas aux genres et aux âges?

Pandas permet de **regrouper** des données avec la méthode `groupby()`.

Voici par exemple le total pour les cantons romands:

In [35]:
dfr.groupby('canton')['nouveaux cas'].sum()

canton
FR    1771
GE    7178
JU     280
NE     847
VD    7520
VS    2334
Name: nouveaux cas, dtype: int64

Mais on peut aussi regrouper par canton **et âge**:

In [36]:
dfr.groupby(['canton', 'age'])['nouveaux cas'].sum()

# Réunir deux jeux de données
On peut ajouter les noms des cantons à l’aide d’un autre tableau de données, qu’on va charger dans la variable **df_cantons**:

In [24]:
df_cantons = pd.read_csv('data/cantons_geo.csv')
df_cantons.head()

,Unnamed: 0,name_fr,name_de,abbrev,lat,lng
0,0,Argovie,Aargau,AG,"47,3876664","8,1554295"
1,1,Appenzell Rhodes-Extérieures,Appenzell Ausserrhoden,AR,"47,366481","9,3000916"
2,2,Appenzell Rhodes-Intérieures,Appenzell Innerrhoden,AI,"47,3161925","9,6316573"
3,3,Bâle-Campagne,Basel-Landschaft,BL,"47,4418122","7,7644002"
4,4,Bâle-Ville,Basel-Stadt,BS,"47,5619253","7,592768"


Accessoirement, ce tableau vous donne les coordonnées géographiques des centres des cantons. Mais on ne va utiliser que leur abréviation (AG) et leur nom en français.

In [38]:
df_cantons = df_cantons[['name_fr', 'abbrev']]

## Renommer la colonne commune

Il va falloir donner à la colonne «abbrev» le nom utilisé dans notre df principal, soit «canton». A vous!

In [56]:
df_cantons.rename(columns={
    'abbrev':'canton'
}, inplace=True)
df_cantons.head(5)

,name_fr,canton
0,Argovie,AG
1,Appenzell Rhodes-Extérieures,AR
2,Appenzell Rhodes-Intérieures,AI
3,Bâle-Campagne,BL
4,Bâle-Ville,BS


Et maintenant, on est prêt à **réunir ces tableaux**! Il faut indiquer à Pandas quelle colonne il doit utiliser pour aligner chaque ligne, et comment gérer les lignes pour lesquelles il n’existe aucune correspondance.

In [57]:
dfm = df.merge(df_cantons, on='canton', how='left')
dfm.head(5)

,date cas,canton,age,genre_num,genre,nouveaux cas,name_fr_x,name_fr_y
0,2020-02-24,TI,70 - 79,1,homme,1,Tessin,Tessin
1,2020-02-27,TI,50 - 59,2,femme,1,Tessin,Tessin
2,2020-02-27,TI,70 - 79,1,homme,2,Tessin,Tessin
3,2020-02-29,TI,50 - 59,2,femme,1,Tessin,Tessin
4,2020-03-01,TI,80+,1,homme,1,Tessin,Tessin


Ici, on a gardé toutes les lignes de **dfr**, mais seulement les lignes de **df_cantons** qui y correspondent. C’est ce qu’on appelle un «left join»: le résultat contient tout le tableau de gauche mais seulement les éléments pertinents du tableau de droite.

Par défaut, la méthode `df.merge()` utilise un «inner join»: elle ne conserve que les éléments qui ont un élément en commun dans la colonne indiquée après `on`.

En l’occurrence, comme tous les cantons de notre **dfr** sont aussi dans **df_cantons**, le résultat du «left join» ne fait pas de différence avec un «inner join».

Cette page [détaille les différents types de «joints»](https://www.datacourses.com/concatenate-merge-and-join-data-with-pandas-598/) qu’on peut utiliser.

On va maintenant faire la même opération sur **df** (Suisse entière), et écraser **df** avec le résultat.

In [49]:
df = df.merge(df_cantons, on='canton')
df.head()

,date cas,canton,age,genre_num,genre,nouveaux cas,name_fr
0,2020-02-24,TI,70 - 79,1,homme,1,Tessin
1,2020-02-27,TI,50 - 59,2,femme,1,Tessin
2,2020-02-27,TI,70 - 79,1,homme,2,Tessin
3,2020-02-29,TI,50 - 59,2,femme,1,Tessin
4,2020-03-01,TI,80+,1,homme,1,Tessin


### Cette fois, on est bon!

On a notre colonne name_fr. On peut transformer le tableau et l’exporter pour en faire par exemple un graphique [Datawrapper](https://www.datawrapper.de).

In [50]:
df

,date cas,canton,age,genre_num,genre,nouveaux cas,name_fr
0,2020-02-24,TI,70 - 79,1,homme,1,Tessin
1,2020-02-27,TI,50 - 59,2,femme,1,Tessin
2,2020-02-27,TI,70 - 79,1,homme,2,Tessin
3,2020-02-29,TI,50 - 59,2,femme,1,Tessin
4,2020-03-01,TI,80+,1,homme,1,Tessin
...,...,...,...,...,...,...,...
16587,2020-08-24,UR,10 - 19,1,homme,1,Uri
16588,2020-08-24,UR,20 - 29,1,homme,1,Uri
16589,2020-08-24,UR,50 - 59,1,homme,1,Uri
16590,2020-08-25,UR,20 - 29,1,homme,1,Uri


## Exercices bonus

Arrivez-vous à joindre notre df actuel avec la population des cantons romands, contenue dans **data/cantons_population.csv** ?

In [54]:
dfm = dfm.merge(df_population, on='name')

KeyError: 'name'

In [52]:
df_population = pd.read_csv('data/cantons_population.csv')
df_population

,name,population
0,Appenzell Rhodes-Extérieures,55234
1,Appenzell Rhodes-Intérieures,16145
2,Argovie,678207
3,Berne,1034977
4,Bâle-Campagne,288132
5,Bâle-Ville,194766
6,Fribourg,318714
7,Genève,499480
8,Glaris,40403
9,Grisons,198379


## Exercice bonus 2

On a vu passer cette drôle de classe d’âge: «Unbekannt»… Comment mettre ça en français?